In [4]:
from urllib.request import urlopen

In [3]:
try:
    html=urlopen("https://www.neo-messtechnik.com/en/power-quality-explained-chapter6-pq-qa")
    bs=BeautifulSoup(html.read(),'html.parser') #html5lib
    print(bs.h1)
except Exception as e:
    print(e)


<h1>Q&amp;A – 12 answers to 10 questions in Chapter 6</h1>


In [4]:
try:
     badContent = bs.nonExistingTag.anotherTag
except AttributeError as e:
     print('Tag was not found')
else:
     if badContent == None:
             print ('Tag was not found')

Tag was not found


C:\Users\simo_\AppData\Local\Temp\ipykernel_28044\1351072771.py:2: DeprecationWarning: .nonExistingTag is deprecated, use .find("nonExisting") instead. If you really were looking for a tag called nonExistingTag, use .find("nonExistingTag")
  badContent = bs.nonExistingTag.anotherTag


In [5]:

qa_dict = {}

questions = bs.find_all('h2')

for question in questions:
    question_text = question.get_text(strip=True)

    answer_text = []

    next_tag = question.find_next_sibling()

    while next_tag and next_tag.name != 'h2':
        if next_tag.name == 'p':
            answer_text.append(next_tag.get_text(strip=True))
        next_tag = next_tag.find_next_sibling()

    full_answer = ' '.join(answer_text)
    
    qa_dict[question_text] = full_answer
    
qa_list = list(qa_dict.items())
qa_list = qa_list[1:-2]
# for q, a in qa_list:
    # print(f"Question: {q}\nRéponse: {a}\n")




In [50]:
import json
# Application du pipeline de prétraitement à chaque contenu extrait
list_results = []
with open('D:\\Generative IA\\rag_pdfs\\Q_A_POWER_QUALITY_12_QUESTION.json', 'w', encoding='utf-8') as f:
    for question, response in qa_list:
        print(question)
        result = {"question":question, "response":response}
        
        json.dump(result, f, ensure_ascii=False)
        f.write("\n")  

        


What do star-, delta- and single-line wiring mean?
What does a Transient Recorder do?
How to spot a highly evolved Power Quality Analyzer
What are ½ period values?
What is the difference between disturbance and transient recording?
What is meant by system dynamics?
What is the function of a PMU?
NEO PQA 8000H
Could you explain the PMU IEEE C 37.118 standard?
What does RoCof stand for?
What are WAMS in smart grids?
What is a Power Quality Spreading & Mitigation Analysis?


In [51]:
# schneider
try:
    html=urlopen("https://www.electrical-installation.org/enwiki/Definition_of_voltage_ranges")
    bs=BeautifulSoup(html.read(),'html.parser') #html5lib
    print(bs.h1)
except Exception as e:
    print(e)


<h1 class="firstHeading" id="firstHeading"><span class="mw-page-title-main">Definition of voltage ranges</span></h1>


In [6]:
#bs.body.get_text()

In [58]:
# from transformers import pipeline

# # Charger le modèle spaCy

# # Charger le modèle Llama 3 pour la génération de texte sur GPU
# pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B",device=0)     
# # Génération des questions
# def generate_questions(content):
#     prompt = (
#         "Based on the following content, generate relevant questions and responses related to the Electrical Field:\n\n"
#         f"{content}\n\n"
#         "Questions and Responses:\n"
#     )

#     # Génération de texte
#     questions = pipe(prompt, max_new_tokens=50, num_return_sequences=1)
#     return questions[0]['generated_text']

# questions=generate_questions(bs.body.get_text())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
import re
import json
from bs4 import BeautifulSoup

model= ChatOllama(model="llama3:8B",temperature = 0)

# template0 = """
# You are an expert in electrical engineering, specializing in the quality of the electrical grid and energy efficiency and Energy management system (EMS). 
# The following content is extracted from a webpage related to electrical engineering. 
# You are tasked with generating question and answer pairs based on this {content}. 
# Please follow these instructions carefully: 
# 1. **Extract Information and Generate Q&A:** From the technical content, generate a clear, insightful question based on each key section of the content and provide a comprehensive, precise answer. Ensure the answers are accurate, complete, and aligned with electrical engineering standards and regulations.
# 2. **Question and Answer Format:** Each response should be in the following format: 
#     - Question: [Generated question from the content]
#     - Answer: [The detailed answer extracted from the content, including definitions, explanations, and examples where applicable]
# 3. **No Extra Content:** Do not include any additional text, comments, or explanations in your response.
# 4. **Focus on Content-Driven Questions:** Ensure that all questions are based strictly on the provided content and do not reference any figures or external materials.
# 5. **Quality Assurance:** Each answer must reflect a deep understanding of electrical engineering principles, aiming to satisfy both novice and advanced readers.
# 6. **Empty Response:** If no question-answer pair can be generated, return an empty string ('').
# 7. **Final Check for Uniqueness:** Before finalizing, ensure all questions are distinct and that each question-answer pair provides new, non-redundant information.
# 8. **No Extra Content:** Do not include any additional text, comments, or explanations in your response. """

template = """
You are an expert in electrical engineering, specializing in the quality of the electrical grid, energy efficiency, and Energy Management Systems (EMS).

The following content is extracted from a webpage related to electrical engineering. Your task is to generate question-and-answer pairs based on this content.

Instructions:

1. **Extract Information and Generate Q&A:** For each key section of the content, generate a clear and insightful question and provide a precise answer. Ensure the answers are accurate and aligned with electrical engineering standards.

2. **Format:** Each response should be in the following format:

   Question: [Generated question from the content]

   Answer: [Detailed answer extracted from the content]

3. **Sequential Generation:** Focus on one section at a time, ensuring each question-answer pair is clearly separated by new lines.

4. **Content-Driven Questions:** Base all questions strictly on the provided content. Do not reference external materials.

5. **Quality Assurance:** Each answer must demonstrate a deep understanding of electrical engineering principles.

6. **No Extra Content:** Do not include any additional text or comments.

7. **Final Check for Uniqueness:** Ensure all questions are distinct and provide new information.

8. **Empty Response:** If no question-answer pair can be generated from a section, return an empty string ('').

Generate the question and answer pairs based on the following content:
{content}
"""



def strparser(response_text):
    pattern = re.compile(r"Question:\s*(.*?)\n+Answer:\s*(.*?)\n+(?=Question:|$)", re.DOTALL)

    qa_list = []

    matches = pattern.findall(response_text)
    i=0
    for match in matches:
      
        print(i)
        question, answer = match
        elem=json.dumps({"question": question.strip(), "answer": answer.strip()})
        qa_list.append(elem)
        i=i+1
    
    return qa_list

def parse_with_ollama(cleaned_content):
    prompt = PromptTemplate(template=template, input_variables=["content",])

    chain = prompt | model
    # Exécuter le modèle sur le contenu fourni
  #  print(cleaned_content)
    response = chain.invoke(
        {"content": cleaned_content}
    )

    # Afficher la réponse générée par le modèle
    print(response)

    return response

def scrape_website(website):
    print("Launching Chrome browser...")
    chrome_driver_path = "D:\\Generative IA\\rag_pdfs\\chromedriver.exe"
    
    # Correctly initialize ChromeOptions
    options = Options()
    
    # Create the Chrome driver
    driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)
    
    try:
        driver.get(website)
        print("Page loaded...")
        html = driver.page_source
        
        # Pause to allow for full page load
        import time
        time.sleep(10)
        
        return html
    finally:
        driver.quit()

def extract_body_content(html_content):
    soup= BeautifulSoup(html_content,'html.parser')
    body_content=soup.body
    if body_content:
        return str(body_content)
def clean_body_content(body_content):
    soup = BeautifulSoup(body_content,'html.parser')
    for script_or_style in soup(["script","style"]):
        script_or_style.extract()
        cleaned_content= soup.get_text(separator="\n")
        cleaned_content= "\n".join(line.strip() for line in cleaned_content.splitlines() if line.strip())
        return cleaned_content
    

def remove_after_retrieved(content):
    cleaned_content = re.sub(r"(Retrieved from.*|https://.*)", "", content, flags=re.DOTALL)
    
    return cleaned_content.strip()
# Example usage
#print(html_content)
#chapter A : General rules of electrical installation design
# urls=["https://www.electrical-installation.org/enwiki/Resistive-type_heating_appliances_and_incandescent_lamps_(conventional_or_halogen)",
#            # "https://www.electrical-installation.org/enwiki/Electrical_installation_design_methodology",
#            # "https://www.electrical-installation.org/enwiki/Electrical_installation_rules,_standards",
#            # "https://www.electrical-installation.org/enwiki/Fluorescent_lamps",
#            # "https://www.electrical-installation.org/enwiki/Discharge_lamps",
#            # "https://www.electrical-installation.org/enwiki/LED_lamps_and_fixtures",
#            # "https://www.electrical-installation.org/enwiki/Installed_power_(kW)",
#            # "https://www.electrical-installation.org/enwiki/Installed_apparent_power_(kVA)",
#            # "https://www.electrical-installation.org/enwiki/Estimation_of_actual_maximum_kVA_demand",
#            # "https://www.electrical-installation.org/enwiki/Example_of_application_of_factors_ku_and_ks",
#            # "https://www.electrical-installation.org/enwiki/Choice_of_transformer_rating"]
           
# Power Factor    
# urls=["https://www.electrical-installation.org/enwiki/Definition_of_Power_Factor"
#      ,"https://www.electrical-installation.org/enwiki/Definition_of_reactive_power",
#       "https://www.electrical-installation.org/enwiki/The_nature_of_reactive_power",
#       "https://www.electrical-installation.org/enwiki/Reactive_power_of_capacitors",
#       "https://www.electrical-installation.org/enwiki/Equipment_and_appliances_requiring_reactive_energy",
#       "https://www.electrical-installation.org/enwiki/Practical_values_of_power_factor",
#       "https://www.electrical-installation.org/enwiki/Why_improve_the_power_factor%3F",
#       "https://www.electrical-installation.org/enwiki/Theoretical_principles_to_improve_power_factor",
#       "https://www.electrical-installation.org/enwiki/Equipment_to_improve_power_factor",
#       "https://www.electrical-installation.org/enwiki/The_choice_between_a_fixed_or_automatically-regulated_bank_of_capacitors",
#       "https://www.electrical-installation.org/enwiki/Where_to_install_power_factor_correction_capacitors%3F",
#       "https://www.electrical-installation.org/enwiki/How_to_determine_the_optimum_level_of_compensation%3F",
#       "https://www.electrical-installation.org/enwiki/Compensation_to_increase_the_available_active_power_output",
#       "https://www.electrical-installation.org/enwiki/Compensation_of_reactive_energy_absorbed_by_the_transformer",
#       "https://www.electrical-installation.org/enwiki/Power_factor_correction_of_induction_motors",

#      ]
    
    # Harmonic
# urls =["https://www.electrical-installation.org/enwiki/Detect_and_eliminate_harmonics:_why%3F",
#        "https://www.electrical-installation.org/enwiki/Definition_of_harmonics",
#        "https://www.electrical-installation.org/enwiki/Origin_of_harmonics",
#        "https://www.electrical-installation.org/enwiki/Harmonic_distortion_indicators_-_Power_factor",
#        "https://www.electrical-installation.org/enwiki/Harmonic_distortion_indicators_-_Crest_factor",
       
#        "https://www.electrical-installation.org/enwiki/Harmonic_spectrum",
#        "https://www.electrical-installation.org/enwiki/R.m.s._values",
#        "https://www.electrical-installation.org/enwiki/Usefulness_of_the_various_indicators_of_Harmonic_distortion",
#        "https://www.electrical-installation.org/enwiki/Harmonic_measurement_in_electrical_networks",
#        "https://www.electrical-installation.org/enwiki/Effects_of_harmonics_-_Resonance",
#        "https://www.electrical-installation.org/enwiki/Effects_of_harmonics_-_Increased_losses",
#        "https://www.electrical-installation.org/enwiki/Effects_of_harmonics_-_Overload_of_equipment",
#        "https://www.electrical-installation.org/enwiki/Effects_of_harmonics_-_Disturbances_affecting_sensitive_loads",
#        "https://www.electrical-installation.org/enwiki/Effects_of_harmonics_-_Economic_impact",
#        "https://www.electrical-installation.org/enwiki/Harmonics_standards",
#        "https://www.electrical-installation.org/enwiki/Basic_solutions_to_mitigate_harmonics",
#        "https://www.electrical-installation.org/enwiki/Harmonic_filtering",
#        "https://www.electrical-installation.org/enwiki/The_method_to_optimize_harmonics_mitigation",
  
# energy effiency
# urls=["https://www.electrical-installation.org/enwiki/Energy_Efficiency_and_decarbonization_in_brief",
#      "https://www.electrical-installation.org/enwiki/An_international_appetite_for_Energy_Efficiency_and_Prosumer_regulations",
#      "https://www.electrical-installation.org/enwiki/Energy_Efficiency_and_Prosumer_electrical_installation_standards",
#      "https://www.electrical-installation.org/enwiki/IEC_60364-8-1_standard:_the_state-of-the-art_for_Energy_Efficiency_in_electrical_installations",
#      "https://www.electrical-installation.org/enwiki/Design_of_Prosumer_electrical_installations",
#      "https://www.electrical-installation.org/enwiki/Diagnosis_through_electrical_measurement","https://www.electrical-installation.org/enwiki/Energy_saving_opportunities",
#      "https://www.electrical-installation.org/enwiki/Smart_Panels",
#       "https://www.electrical-installation.org/enwiki/How_to_evaluate_energy_savings",
#       "https://www.electrical-installation.org/enwiki/What_does_self-consumption_mean%3F",
#       "https://www.electrical-installation.org/enwiki/Types_of_Prosumer_electrical_installation_according_to_IEC_60364-8-82",
#       "https://www.electrical-installation.org/enwiki/Architectures_for_integration_of_local_sources",
#       "https://www.electrical-installation.org/enwiki/Calculation_rules_when_integrating_solar_and_storage",
#       "https://www.electrical-installation.org/enwiki/Sizing_of_electrical_installation_with_solar_production",
#       "https://www.electrical-installation.org/enwiki/Specific_protection_of_prosumer_electrical_installations",
#       "https://www.electrical-installation.org/enwiki/Power_factor_-_impact_of_solar_self-consumption",
#       "https://www.electrical-installation.org/enwiki/Excess_local_production_-_how_to_manage",
#      ]
#Protections
# urls=["https://www.electrical-installation.org/enwiki/Protection_against_fire_due_to_earth_faults",
      
#       "https://www.electrical-installation.org/enwiki/Types_of_protection_against_electric_shock",
#       "https://www.electrical-installation.org/enwiki/Additional_protection:_High_sensitivity_RCDs",
#       "https://www.electrical-installation.org/enwiki/TT_system_-_Principle",
#       "https://www.electrical-installation.org/enwiki/TT_system_-_Practical_aspects",
#       "https://www.electrical-installation.org/enwiki/TN_system_-_Principle",
#       "https://www.electrical-installation.org/enwiki/TN_system_-_Preliminary_conditions",
      
#       "https://www.electrical-installation.org/enwiki/TN_system_-_Earth-fault_current_calculation",
#       "https://www.electrical-installation.org/enwiki/TN_system_-_High_fault_current-loop_impedance",
#       "https://www.electrical-installation.org/enwiki/IT_system_-_Principle",
#       "https://www.electrical-installation.org/enwiki/IT_system_-_Fault_protection",
#       "https://www.electrical-installation.org/enwiki/IT_system_-_Implementation_of_protections",
#       "https://www.electrical-installation.org/enwiki/IT_system_-_Practical_aspects",
#       "https://www.electrical-installation.org/enwiki/Description_of_RCDs",
#       "https://www.electrical-installation.org/enwiki/Form_factors_of_RCDs",
#       "https://www.electrical-installation.org/enwiki/Types_of_RCDs",
#       "https://www.electrical-installation.org/enwiki/Sensitivity_of_RCDs_to_disturbances",
#       "https://www.electrical-installation.org/enwiki/Coordination_of_residual_current_protective_devices",
#       "https://www.electrical-installation.org/enwiki/RCDs_selection_in_presence_of_DC_earth_leakage_currents",
#       "https://www.electrical-installation.org/enwiki/Extra_Low_Voltage_(ELV)",
      
#       "https://www.electrical-installation.org/enwiki/The_electrical_separation_of_circuits",
#       "https://www.electrical-installation.org/enwiki/Class_II_equipment",
#       "https://www.electrical-installation.org/enwiki/Out-of-arm%E2%80%99s_reach_or_interposition_of_obstacles",
#       "https://www.electrical-installation.org/enwiki/Protection_against_arc_faults_in_cables_and_connections_(AFDD)",
#       "https://www.electrical-installation.org/enwiki/Protection_against_electrical_fire_risks_-_synthesis"]
      

      
      
      
 # Power supllay     
# urls=["https://www.electrical-installation.org/enwiki/Power_supply_at_medium_voltage",
#     "https://www.electrical-installation.org/enwiki/Substation_including_generators_and_parallel_operation_of_transformers",
#     "https://www.electrical-installation.org/enwiki/Medium_voltages_and_current_values_according_to_IEC_Standards",
#     "https://www.electrical-installation.org/enwiki/Different_types_of_MV_power_supply",
#     "https://www.electrical-installation.org/enwiki/Some_practical_issues_concerning_MV_distribution_networks",
#     "https://www.electrical-installation.org/enwiki/Procedure_for_the_establishment_of_a_new_substation",
#     "https://www.electrical-installation.org/enwiki/General_principle_of_protection_against_electrical_shocks_in_electrical_installations",
#     "https://www.electrical-installation.org/enwiki/Protection_of_transformer_and_circuits",
#     "https://www.electrical-installation.org/enwiki/MV/LV_transformer_protection_with_circuit_breaker",
#     "https://www.electrical-installation.org/enwiki/Interlocks_and_conditioned_operations",
#     "https://www.electrical-installation.org/enwiki/Functions_of_a_substation_with_LV_metering",
#     "https://www.electrical-installation.org/enwiki/Choice_of_MV_equipment_for_a_substation_with_LV_metering",
#     "https://www.electrical-installation.org/enwiki/Functions_of_the_substation_with_MV_metering",
#     "https://www.electrical-installation.org/enwiki/Choice_of_MV_equipment_for_a_substation_with_MV_metering",
#     "https://www.electrical-installation.org/enwiki/Choice_of_MV_equipment",
#     "https://www.electrical-installation.org/enwiki/Instructions_for_use_of_MV_equipment",
#     "https://www.electrical-installation.org/enwiki/Choice_of_MV/LV_transformer",
#     "https://www.electrical-installation.org/enwiki/Ventilation_in_MV_Substations",
#     "https://www.electrical-installation.org/enwiki/Indoor_substation",
#     "https://www.electrical-installation.org/enwiki/Outdoor_substations"]
      
    
#residual

# urls=["https://www.electrical-installation.org/enwiki/Bathroom_electrical_installation",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Distribution_boards_components",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Protection_of_people",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Circuits",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Protection_against_overvoltages_and_lightning",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Periodic_control",
#       "https://www.electrical-installation.org/enwiki/Residential_electrical_installation_-_Requirements_and_recommendations_for_Kitchen",
#       "https://www.electrical-installation.org/enwiki/Requirements_applicable_to_special_installations_and_locations",
      
      
#      ]
      
## ElectroMagnetic

      
# urls = [
#     "https://www.electrical-installation.org/enwiki/EMC_behaviour_of_different_electrical_distribution_architecturing",
#     "https://www.electrical-installation.org/enwiki/EMC_-_Earthing_principles_and_structures",
#     "https://www.electrical-installation.org/enwiki/Equipotential_bonding_inside_and_outside_buildings",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Separating_cables",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Raised_floors",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Cable_running",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Busway",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Implementation_of_shielded_cables",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Communication_networks",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Cabinet_cabling",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Standards",
#     "https://www.electrical-installation.org/enwiki/EMC_implementation_-_Electrostatic_discharge_protection",
#     "https://www.electrical-installation.org/enwiki/Common-mode_impedance_coupling",
#     "https://www.electrical-installation.org/enwiki/Capacitive_coupling",
#     "https://www.electrical-installation.org/enwiki/Inductive_coupling",
#     "https://www.electrical-installation.org/enwiki/Radiated_coupling",
#     "https://www.electrical-installation.org/enwiki/Wiring_recommendations"
# ]

#photovoltaic background

# urls=["https://www.electrical-installation.org/enwiki/Photovoltaic_background,_technology",
#       "https://www.electrical-installation.org/enwiki/PV_System:_how_to_ensure_safety_during_normal_operation",
#       "https://www.electrical-installation.org/enwiki/PV_System:_protection_against_overvoltages",
#       "https://www.electrical-installation.org/enwiki/PV_System:_how_to_ensure_safety_during_maintenance_or_emergency",
#       "https://www.electrical-installation.org/enwiki/PV_System:_how_to_ensure_safety_during_all_the_life_cycle_of_the_installation",
#       "https://www.electrical-installation.org/enwiki/Photovoltaic_architectures_-_common_characteristics",
#       "https://www.electrical-installation.org/enwiki/Photovoltaic_system_architectures",
#       "https://www.electrical-installation.org/enwiki/Photovoltaic_system_sizing",
#       "https://www.electrical-installation.org/enwiki/Photovoltaic_installation_type",
#       "https://www.electrical-installation.org/enwiki/Photovoltaic_system:_electrical_equipments_selection",
#      ]
      
       
       
       
       
       
       
       
# urls=["https://powerqualityanalysers.com/knowledgebase/what-is-the-iec61000-4-30-standard-for-power-quality-analysers/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-class-b-power-quality-analysers/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-class-s-power-quality-analysers/",
#      "https://powerqualityanalysers.com/knowledgebase/what-causes-flicker/",
#      "https://powerqualityanalysers.com/knowledgebase/what-level-of-flicker-can-cause-problems/",
#      "https://powerqualityanalysers.com/knowledgebase/how-is-flicker-measured/",
#      "https://powerqualityanalysers.com/knowledgebase/what-light-sources-can-be-effected-by-flicker/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-power-quality-flicker/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-harmonic-distortions-and-their-effects-and-how-to-measurements-and-analyse/",
#      "https://powerqualityanalysers.com/knowledgebase/what-can-cause-flickering-light-and-voltage-fluctuations/",
#      "https://powerqualityanalysers.com/knowledgebase/how-do-flexible-current-clamps-work/",
#      "https://powerqualityanalysers.com/knowledgebase/what-can-hall-effect-cts-measure/",
#      "https://powerqualityanalysers.com/knowledgebase/do-i-need-current-clamps-to-measure-ac-and-dc-current/",
#      "https://powerqualityanalysers.com/knowledgebase/how-does-a-current-clamp-work/",
#      "https://powerqualityanalysers.com/knowledgebase/can-thdf-and-thdr-give-different-measurements/",
#      "https://powerqualityanalysers.com/knowledgebase/how-many-harmonics-are-used-to-calculate-the-total-harmonic-distortion/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-total-harmonic-distortion-thd/",
#      "https://powerqualityanalysers.com/knowledgebase/why-do-the-zero-sequence-or-triplen-harmonics-case-problems/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-the-most-undesirable-harmonics/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-the-harmonics-characteristics-in-three-phase-systems/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-the-correct-sample-period-when-performing-a-power-quality-survey/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-the-correct-sample-period-when-looking-for-power-quality-problems/",
#      "https://powerqualityanalysers.com/knowledgebase/how-much-long-do-i-need-to-make-power-quality-measurements-for/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-secr/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-crc/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-the-difference-between-power-factor-and-displaced-power-factor/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-apparent-power/",
#      "https://powerqualityanalysers.com/knowledgebase/why-do-industrial-users-need-to-measure-reactive-power/",
#      "https://powerqualityanalysers.com/knowledgebase/do-domestic-installations-need-to-measure-reactive-power/",
#      "https://powerqualityanalysers.com/knowledgebase/why-do-i-need-to-measure-reactive-power/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-reactive-power-and-nonactive-power/",
#      "https://powerqualityanalysers.com/knowledgebase/what-do-we-mean-by-an-unbalanced-system/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-a-voltage-swell/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-a-voltage-interruption/",
#      "https://powerqualityanalysers.com/knowledgebase/what-is-a-voltage-dip/",
#      "https://powerqualityanalysers.com/knowledgebase/what-are-voltage-dips-swells-and-interruptions/",]
      
# QA ems level 1
urls=["https://www1.eere.energy.gov/manufacturing/eguide/foundational_energy_management.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_1_2.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_1_3.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_1_4.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_1_5.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_2.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_3.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_4.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_5.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_6.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_7.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_8.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_9.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_10.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_2_11.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_3_1.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_3_2.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_3_4.html",
      "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_3_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_3.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_4.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_6.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_7.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_8.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_1_9.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_3.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_4.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_6.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_7.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_8.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_9.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_10.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_2_11.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_3.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_4.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_6.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_7.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_3_8.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_4_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_4_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_4_3.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_4_4.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_4_5.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_5_1.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_5_2.html",
     "https://www1.eere.energy.gov/manufacturing/eguide/iso_step_5_3.html",
    "https://www1.eere.energy.gov/manufacturing/eguide/foundational_step_3.html",
]
      
      
def split_dom_content(dom_content,max_length=5000):
    return [ dom_content[i:i+ max_length] for i in range(0,len(dom_content),max_length)]    
    
           
           
      
      
      
    
           
           
           
           

In [23]:
print(len(urls))

59


In [24]:

with open('D:\\Generative IA\\rag_pdfs\\Q_A_ems_level1_2.json', 'w', encoding='utf-8') as f:

            for url in urls:

                    html = scrape_website(url)
                    body=extract_body_content(html)
                    cleaned_content=clean_body_content(body)
                    cleaned_content=cleaned_content.replace('\n',' ')
                    cleaned_content=remove_after_retrieved(cleaned_content)
                    splited_documents=split_dom_content(cleaned_content)
                    for doc in splited_documents:
                        response=parse_with_ollama(doc)
                        qa_list=strparser(response.content)
                        for elem in qa_list:
                            f.write(elem)
                            f.write("\n")  
                        print("############")
    



Launching Chrome browser...
Page loaded...
content='Here are the question-and-answer pairs based on the provided content:\n\nQuestion: What is eGuide Level 1, and what is its purpose?\n\nAnswer: eGuide Level 1 is the Foundational Energy Management level, designed to help implement a basic energy management program at organizations of any size and type. Its purpose is to establish a continual improvement approach to implementing energy projects and sustaining the energy savings achieved.\n\nQuestion: What is the main difference between eGuide Level 1 and eGuide Level 2?\n\nAnswer: The main difference is that eGuide Level 1 focuses on developing an energy management program with a continuous approach to implementing energy projects, whereas eGuide Level 2 (ISO 50001) transitions the energy management program into an energy management system (EnMS), embedding energy management into daily operations.\n\nQuestion: How long does it take to work through the eGuide Level 1 for ISO 50001 implem

In [37]:
import re
import json

def strparser(response_text):
    # Mise à jour du motif pour capturer plusieurs lignes et inclure des retours à la ligne
    pattern = re.compile(r"Question:\s*(.*?)\n+Answer:\s*(.*?)\n+(?=Question:|$)", re.DOTALL)

    qa_list = []

    matches = pattern.findall(response_text)
    for i, match in enumerate(matches):
        question, answer = match
        elem = json.dumps({"question": question.strip(), "answer": answer.strip()})
        qa_list.append(elem)
    
    return qa_list

# Exemple d'utilisation
response_text = """"Question: What is the primary consideration for selecting energy opportunities for implementation in ISO 50001 (Level 2) foundational energy management?\n\nAnswer: In ISO 50001 (Level 2), one of the chief considerations for implementing energy opportunities is to determine if the opportunity will contribute to improvement of your organization's energy performance and align with other organizational priorities. This involves evaluating factors such as funding availability, personnel time, expertise, and resource requirements, while also considering how the opportunity may affect other aspects of the organization. Additionally, it is essential to assess whether the selected opportunities will help achieve the energy objectives and targets identified by the organization.\n\nQuestion: What are some key considerations for selecting energy opportunities for implementation in ISO 50001 (Level 2) foundational energy management?\n\nAnswer: When selecting an opportunity for implementation in ISO 50001 (Level 2), several factors should be taken into account, including:\n\n1. Energy objective(s) and target(s) determined in Step 2.9\n2. Organizational constraints such as:\n   - Funding availability\n   - Disruption to organization\n   - Availability of labor and expertise\n   - Availability of resources\n\nThese considerations help ensure that the selected opportunities align with the organization's energy objectives, are feasible to implement, and do not disrupt critical operations or require significant resource allocation.\n\nQuestion: What is the purpose of the ENERGY STAR Guidelines for Energy Management?\n\nAnswer: The ENERGY STAR Guidelines for Energy Management provide a step-by-step roadmap for continuous improvement in energy management. These guidelines are based on best practices compiled by the EPA ENERGY STAR Program and offer guidance specific to Step 3 of the eGuide, which focuses on implementing energy-saving measures.\n\nQuestion: What is the Continuous Energy Improvement (CEI) system provided by the Northwest Energy Efficiency Alliance (NEEA)?\n\nAnswer: The CEI system developed by NEEA provides a methodology and tools to support achieving energy management goals and objectives through progressive steps. This system helps organizations implement energy-efficient practices and measure their progress, ultimately contributing to improved energy performance.\n\nQuestion: What is the purpose of the AMO eGuide?\n\nAnswer: The Advanced Manufacturing Office (AMO) eGuide is a resource provided by the Department of Energy's Advanced Manufacturing Office. Its primary goal is to assist organizations in implementing foundational energy management practices, as outlined in ISO 50001 (Level 2), and provide guidance on continuous improvement through progressive steps.\n\nQuestion: What is the significance of the Creative Commons Attribution-ShareAlike 4.0 International License?\n\nAnswer: The content of the AMO eGuide is licensed under a Creative Commons Attribution-ShareAlike 4.0 International License, which allows for free use, sharing, and adaptation of the material, as long as proper attribution is given to the original source.\n\nQuestion: What is the role of the Department of Energy's Advanced Manufacturing Office?\n\nAnswer: The Department of Energy's Advanced Manufacturing Office (AMO) is responsible for providing resources and guidance on advanced manufacturing practices, including energy management. The AMO eGuide serves as a key resource for organizations seeking to implement foundational energy management practices and improve their overall energy performance.\n\nQuestion: What is the purpose of Step 2.10 in the AMO eGuide?\n\nAnswer: Step 2.10 of the AMO eGuide involves selecting energy opportunities for implementation, taking into account factors such as energy objectives and targets, organizational constraints, and resource availability. This step is crucial in identifying the most suitable opportunities for improvement and ensuring that they align with the organization's overall goals.\n\nQuestion: What is the significance of Step 2.10.1 in the AMO eGuide?\n\nAnswer: Step 2.10.1 of the AMO eGuide provides a specific task related to selecting energy opportunities for implementation, which involves evaluating the factors mentioned earlier (energy objectives and targets, organizational constraints, funding availability, personnel time, expertise, and resource requirements). This step ensures that organizations carefully consider these factors when choosing opportunities for improvement.\n\nQuestion: What is the purpose of Step 3 in the AMO eGuide?\n\nAnswer: Step 3 of the AMO eGuide focuses on implementing energy management measures. It provides guidance on how to put the foundational energy management practices into action, ensuring that organizations can effectively implement their selected energy-saving opportunities and achieve their energy objectives.\n\nQuestion: What is the purpose of Step 4 in the AMO eGuide?\n\nAnswer: Step 4 of the AMO eGuide focuses on measuring results. It provides guidance on how to track and measure the effectiveness of the implemented energy management measures, ensuring that organizations can assess their progress and make adjustments as needed.\n\nQuestion: What is the purpose of Step 5 in the AMO eGuide?\n\nAnswer: Step 5 of the AMO eGuide emphasizes the importance of reviewing and improving energy management practices continuously. It encourages organizations to regularly review their progress, identify areas for improvement, and implement changes to further enhance their energy performance.\n\nQuestion: What is the purpose of Step 2.11 in the AMO eGuide?\n\nAnswer: Step 2.11 of the AMO eGuide involves preparing an action plan for implementing selected energy opportunities. This step requires organizations to outline the specific actions they will take, the resources required, and the timeline for implementation.\n\nQuestion: What is the purpose of Step 4 in the AMO eGuide?\n\nAnswer: There is no information provided about Step 4 in the AMO eGuide.\n\nQuestion: What is the purpose of Step 5 in the AMO eGuide?\n\nAnswer: There is no information provided about Step 5 in the AMO eGuide.\n\nSince there are no more questions, I will provide a final answer:\n\nThere is no final question."."""

# Appel de la fonction
qa_list = strparser(response_text)
for qa in qa_list:
    print(qa)


{"question": "What is the primary consideration for selecting energy opportunities for implementation in ISO 50001 (Level 2) foundational energy management?", "answer": "In ISO 50001 (Level 2), one of the chief considerations for implementing energy opportunities is to determine if the opportunity will contribute to improvement of your organization's energy performance and align with other organizational priorities. This involves evaluating factors such as funding availability, personnel time, expertise, and resource requirements, while also considering how the opportunity may affect other aspects of the organization. Additionally, it is essential to assess whether the selected opportunities will help achieve the energy objectives and targets identified by the organization."}
{"question": "What are some key considerations for selecting energy opportunities for implementation in ISO 50001 (Level 2) foundational energy management?", "answer": "When selecting an opportunity for implementa

In [94]:
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.common.by import By
AUTH = 'brd-customer-hl_a1336bfd-zone-scraping_browser1:au82x8lz0k54'
SBR_WEBDRIVER = f'https://{AUTH}@zproxy.lum-superproxy.io:9515'

def scrape_website(website):
    print("Launching Chrome browser...")
    print('Connecting to Scraping Browser...')
    sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
    with Remote(sbr_connection, options=ChromeOptions()) as driver:
        print('Connected! Navigating...')
        driver.get(website)
        # solve_res= driver.execute('executeCdpCommand',{
        #     'cmd':'Captcha.waitForSolve','params':{'detectTimout':1000}})
        # print('Captcha solve status:',solve_res['value']['status'])
        print('Taking page screenshot to file page.png')
        print('Navigated! Scraping page content...')
        html = driver.page_source
        return html
        

html=scrape_website("https://www.electrical-installation.org/enwiki/Induction_motors")


        

Launching Chrome browser...
Connecting to Scraping Browser...
Connected! Navigating...
Taking page screenshot to file page.png
Navigated! Scraping page content...


In [94]:
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.common.by import By
AUTH = 'brd-customer-hl_a1336bfd-zone-scraping_browser1:au82x8lz0k54'
SBR_WEBDRIVER = f'https://{AUTH}@zproxy.lum-superproxy.io:9515'

def scrape_website(website):
    print("Launching Chrome browser...")
    print('Connecting to Scraping Browser...')
    sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
    with Remote(sbr_connection, options=ChromeOptions()) as driver:
        print('Connected! Navigating...')
        driver.get(website)
        # solve_res= driver.execute('executeCdpCommand',{
        #     'cmd':'Captcha.waitForSolve','params':{'detectTimout':1000}})
        # print('Captcha solve status:',solve_res['value']['status'])
        print('Taking page screenshot to file page.png')
        print('Navigated! Scraping page content...')
        html = driver.page_source
        return html
        

html=scrape_website("https://www.electrical-installation.org/enwiki/Induction_motors")


        

Launching Chrome browser...
Connecting to Scraping Browser...
Connected! Navigating...
Taking page screenshot to file page.png
Navigated! Scraping page content...


In [135]:
def extract_body_content(html_content):
    soup= BeautifulSoup(html_content,'html.parser')
    body_content=soup.body
    if body_content:
        return str(body_content)
def clean_body_content(body_content):
    soup = BeautifulSoup(body_content,'html.parser')
    for script_or_style in soup(["script","style"]):
        script_or_style.extract()
        cleaned_content= soup.get_text(separator="\n")
        cleaned_content= "\n".join(line.strip() for line in cleaned_content.splitlines() if line.strip())
        return cleaned_content
    
def split_dom_content(dom_content,max_length=60000):
    return [ dom_content[i:i+ max_length] for i in range(0,len(dom_content),max_length)]


def remove_after_retrieved(content):
    cleaned_content = re.sub(r"(Retrieved from.*|https://.*)", "", content, flags=re.DOTALL)
    
    return cleaned_content.strip()

In [136]:
body=extract_body_content(html)
cleaned_content=clean_body_content(body)
split_dom_content=split_dom_content(cleaned_content)
cleaned_content=cleaned_content.replace('\n',' ')
cleaned_content=remove_after_retrieved(cleaned_content)
cleaned_content